# EnergyAnalysis Class
Calculate the solar supply, demand, storage left, and storage demand with given data of area (default as California)
...

## Import packages

In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import h5pyd

## Class implementation

### Energy Supply Analysis

In [2]:
class EnergySupplyAnalysis:
    '''
    The class for energy supply analysis for given capacity and GHI data

    Parameter
    ---------
    capacity: int
        Utility-scale solar power capacity
        unit: Mega Watts
    
    solarDataFile: str
        Solar data file path
    '''
    def __init__(self, f: h5pyd._hl.files.File, capacity=17500) -> None:
        assert isinstance(capacity, float) or isinstance(capacity, int)
        assert isinstance(f, h5pyd._hl.files.File)
        
        self.capacity = capacity
        dset = f['ghi']
        factor = f['ghi'].attrs['psm_scale_factor']
        self.__solarData = pd.DataFrame()
        self.__solarData['Date'] = pd.to_datetime(f['time_index'][...].astype(str))
        self.__solarData['GHI'] = dset / factor
        self.__solarData['Supply'] = self.__solarData['ghi'] * capacity / 1000.
    
    def getEnergySupply(self) -> pd.DataFrame:
        return self.__solarData

### Energy Demand Analysis

In [9]:
class EnergyDemandAnalysis:
    '''
    The class for energy demand analysis for given electrical data

    Parameter
    ---------
    energyDataFile: str
        Electrical demand data file path
    '''
    def __init__(self, energyDataFile: str='../raw_data/CAISOactualLoad.csv') -> None:
        assert isinstance(energyDataFile, str)

        self.demandData = pd.read_csv(energyDataFile)
        self.__energyDemand()

    def __energyDemand(self) -> pd.DataFrame:
        '''
        Calculate the demand with electrical data

        Returns
        -------
        pandas.DataFrame
            | Date | Demand |
            |------|--------|
        
            Date: datetime
            Demand: float
        '''

        data = self.demandData[self.demandData['zone'] == 'CA ISO']
        data['date'] = pd.to_datetime(data['Date'])
        self.__demandDF = data[['date', 'load']]
        self.__demandDF = self.__demandDF.rename(columns={"date": "Date", "load": "Demand"})
    
    def getEnergyDemand(self) -> pd.DataFrame:
        return self.__demandDF

In [90]:
class testAnalysis:
    '''
    Write sth...
    '''
    def __init__(self, supply_df, demand_df) -> None:
        assert isinstance(supply_df, pd.DataFrame)
        assert isinstance(demand_df, pd.DataFrame)

        self.supply_df = supply_df
        self.demand_df = demand_df
        self.__storageAnalysis()

    def __storageAnalysis(self) -> pd.DataFrame:
        '''
        Calculate the supply, demand, storageLeft, storageDemand with given data

        Returns
        -------
        pandas.DataFrame
            | date | hour | supply | demand | storageLeft | storageDemand |
            |------|------|--------|--------|-------------|---------------|
        
            date: str
                %m/%d/%Y
                04/12/2018
            hour: int
            supply: float
            demand: float
            storageLeft: float
            storageDemand: float
        '''
        self.supply_df = self.supply_df.set_index(['Year', 'Month', 'Day', 'Hour'])
        self.demand_df = self.demand_df.set_index(['Year', 'Month', 'Day', 'Hour'])
        print(self.demand_df)
        self.__energyDF = pd.concat([self.supply_df, self.demand_df], axis=1, join='inner')
        # print(self.__energyDF)
        # self.__energyDF = outputs
    
    def getStorageDemand(self) -> pd.DataFrame:
        return self.__energyDF['storageDemand']




In [ ]:
#Energy storage analysis:

class StorageAnalysis:
    """
        This class takes in a pandas dataframe in the form of 
            | date | hour | supply |
            |------|------|--------|
            and
            | date | hour | demand |
            |------|------|--------|

            and creates a dataframe in the form of:
            
            | date | hour | storage demand | storage supplied | storageLeft |
            |------|------|----------------|------------------|-------------|

            where storage demand is the need for power that is not met by solar capacity,
            storage supplied is the amount of energy supplied that hour, which should equal demand unless storageLeft goes to zero.
            storageLeft is a the amount of energy left in the storage devices.
            
    """
    def __init__(self, the_supply: pd.DataFrame, the_demand: pd.DataFrame, the_storageMWh: int):
        self.supply = the_supply
        self


## Test